<a href="https://colab.research.google.com/github/sujan6539/ml/blob/main/MNIST_CNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import datasets
from torchvision.utils import make_grid


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import confusion_matrix

In [35]:
#Convert MNIST image to tensor of 4 dimensions:
# number of image, height, width, color channel
from torchvision import transforms

transform = transforms.ToTensor()

In [36]:


# pull the mnist data from torchvision that is used for training
# This line imports the MNIST dataset class from the torchvision.datasets module.
train_data = datasets.MNIST(root="./cnn", train = True , download = True, transform = transform)

In [37]:
test_data = datasets.MNIST(root="./cnn", train = False, download = True, transform= transform)

In [38]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./cnn
    Split: Train
    StandardTransform
Transform: ToTensor()

In [39]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./cnn
    Split: Test
    StandardTransform
Transform: ToTensor()

In [45]:
#The DataLoader class is used to create an iterable object
# that can be used to iterate over a dataset in batches.

from torch.utils.data import DataLoader

train_loader = DataLoader(train_data, batch_size = 10, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 10, shuffle= False)

In [47]:
# Define CNNs for example create 2 CNNs

conv1 = nn.Conv2d(in_channels = 1, out_channels = 6, kernel_size = 3, stride = 1)
conv2 = nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size = 3, stride = 1)

In [69]:
# check train_data on the first position
# it will be [position, imagevalue, target]
train_data[2]

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 

In [64]:
#grab the first item from train_data
# separate train_data into i  and x_train ie: image and y_train ie: target.
for i, (x_train, y_train) in enumerate(train_data):
  break

In [67]:
 #print i
 i

0

In [66]:
#print x_train data type
x_train.dtype

torch.float32

In [65]:
y_train

5

In [70]:
x_train.shape # gives 3d batch

torch.Size([1, 28, 28])

In [75]:
#changing x_train to 4d batch
x = x_train.view(1,1,28,28) # 1 batch of 1 image of 28x 28

In [76]:
# pass x_train through convulation layer and reassign into x
x = F.relu(conv1(x))

In [79]:
x.shape
# after conv1, 1 batch gives 6 image of 26x26
#since nn.conv2d asked for 6 out_features


torch.Size([1, 6, 26, 26])

In [80]:
# pass through a first pooling layer
x = F.max_pool2d(input = x, kernel_size = 2, stride = 2)

In [81]:
x.shape

# Note: image is downsized to 13x13 since the kernel_size = 2

torch.Size([1, 6, 13, 13])

In [82]:
x = F.relu(conv2(x))

In [83]:
x.shape

torch.Size([1, 16, 11, 11])

In [84]:
x = F.max_pool2d(input = x, kernel_size = 2, stride = 2)

In [85]:
x.shape

torch.Size([1, 16, 5, 5])

In [86]:
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    # Define CNNs for example create 2 CNNs
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 6, kernel_size = 3, stride = 1)
    self.conv2 = nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size = 3, stride = 1)
    self.fc1 = nn.Linear(in_features = 5*5*16, out_features = 120)
    self.fc2 = nn.Linear(in_features = 120, out_features = 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, X):
    #first pass
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(input = X, kernel_size = 2, stride = 2)
    # second CNNs pass
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(input = X, kernel_size = 2, stride = 2)
    # re-view data to flatten out
    #The view() method in PyTorch is used to reshape a tensor to a new shape.
    # The -1 argument in the view() method tells PyTorch to automatically infer
    # the size of the missing dimension based on the other dimensions and the
    #total number of elements in the tensor.
    #In the given example, X.view(-1, 16*5*5) reshapes the tensor X to a new
    #shape where the first dimension has an unknown size and the second
    # dimension has a size of 16*5*5.
    X = X.view(-1, 16*5*5) # -1 so that we can vary the batch size

    # fully connected layer
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)

    return X


In [88]:
torch.manual_seed(41)

cnn  = ConvolutionalNetwork()
cnn

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [89]:
# loss function optimizer
criterion = nn.CrossEntropyLoss()

"""
The torch.optim.Adam class in PyTorch is an implementation of the
Adam optimizer algorithm. The Adam optimizer is a popular optimization
algorithm that is used to update the weights of a neural network.
"""
optimizer = torch.optim.Adam(params = cnn.parameters(), lr = 0.01)

In [ ]:
import time


start_time = time.time()

# create variables to track things
epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

# create for loop for Epochs
for i in range(epochs):
  trn_correct = 0
  tst_correct = 0

  for b, (x_train, y_train) in enumerate (train_loader):
    # start the batch at 1
    b+=1



# train the data



# test



current_time = time.time()
total_time = current_time - start_time
print(f'Training took: {total/60} minutes')